# Idiomatic Polars 

## Matt Harrison - PyData NYC 2024

## https://github.com/mattharrison/pydata_nyc_2024



<!-- https://github.com/mattharrison/talks>

## About Matt  Harrison @\_\_mharrison\_\_

* Author of *Effective Polars*, *Effective Pandas*, *Effective XGBoost*, *Learning Python for Data*, *Machine Learning Pocket Reference*, and *Illustrated Guide to Python 3*
* Advisor and consultant.
* Corporate trainer at MetaSnake. Taught Pandas to 1000's of students.


## Relevant Background

* 1999 NLP
* 2006 Created Python OLAP Engine
* 2009 Heard about Pandas
* Used Pandas for failure modeling, analytics, and ml
* 2016 Learning the Pandas Library
* 2019 Spark
* 2020 Pandas Cookbook
* 2021 Effective Pandas
* 2022 CuDf, Modin, Polars
* 2023 Effective Pandas 2
* 2024 Effective Polars
* 2024 Effective Polars 1.0
* 2024 NVidia Polars backend


## Why Tabular?

- Deep learning and video/audio are popular but the crown jewels are in Excel or SQL.

- My focus on tabular tooling (Pandas, Polars, XGBoost, CatBoost, etc)


## Sad News

- Python is slow
- Pandas gets around this with NumPy (v1) and PyArrow (v2)
- Polars gets around this with Arrow (Rust)
- Stay in the playground

## Outline of Opinions

* Load Data
* Types
* Chaining
* Apply
* Aggregation

## Polars Overview

* Polars is a Rust library with Python bindings
* Polars has expressions and contexts
  - Contexts - `.select`, `.with_columns`, `.filter`
  - Expressions - done w/ `pl.col("col_name")` or `pl.lit(1)`
* Lazy evaluation
* Query optimization
* Multi-threaded


## Data

In [ ]:
!pip install -U polars

In [ ]:
!uv add gpxpy

In [61]:
import gpxpy
import numpy as np
import polars as pl
import polars.selectors as cs
import pandas as pd

In [62]:
pl.__version__

'1.12.0'

In [63]:
import polars as pl
import gpxpy
import numpy as np

def gpx_to_polars(fname):
    # Parse the GPX file
    data = gpxpy.parse(open(fname))
    prev = None
    data_dict = {'course': [],
                 'distance_2d': [],
                 'latitude': [],
                 'longitude': [],
                 'time': [],
                 'elevation': [],
                 'speed_between': [],
                }
    
    # Iterate through tracks, segments, and points
    for track in data.tracks:
        for seg in track.segments:
            for i, pt in enumerate(seg.points):
                if prev is None:
                    prev = pt
                for key in data_dict:
                    attr = getattr(pt, key)
                    if callable(attr):
                        data_dict[key].append(attr(prev))
                    else:
                        data_dict[key].append(attr)
                prev = pt

    # Create a Polars DataFrame
    df = (pl.DataFrame(data_dict)

        #.with_columns([pl.col("time").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%SZ", strict=False)])
        .with_columns(
            travelled= pl.col("distance_2d").cum_sum(),
            elapsed=(pl.col("time") - pl.col("time").min()).dt.total_seconds()
        )
        .with_columns(        
            avg_velocity=pl.col("travelled") / pl.col("elapsed"),
            rolling_travelled=pl.col("travelled").rolling_mean(window_size=5),
            rolling_elapsed=pl.col("elapsed").rolling_mean(window_size=5),
        )
        .with_columns(
            rolling_velocity=pl.col("rolling_travelled") / pl.col("rolling_elapsed"),
            rolling_between=pl.col("speed_between").rolling_mean(window_size=5),
        )
    )

    return df

df = gpx_to_polars('Face_plant.gpx')
print(df)


shape: (10_430, 14)
┌────────┬────────────┬───────────┬────────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ course ┆ distance_2 ┆ latitude  ┆ longitude  ┆ … ┆ rolling_t ┆ rolling_e ┆ rolling_v ┆ rolling_b │
│ ---    ┆ d          ┆ ---       ┆ ---        ┆   ┆ ravelled  ┆ lapsed    ┆ elocity   ┆ etween    │
│ null   ┆ ---        ┆ f64       ┆ f64        ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│        ┆ f64        ┆           ┆            ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64       │
╞════════╪════════════╪═══════════╪════════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ null   ┆ 0.0        ┆ 40.879161 ┆ -111.85516 ┆ … ┆ null      ┆ null      ┆ null      ┆ null      │
│        ┆            ┆           ┆ 9          ┆   ┆           ┆           ┆           ┆           │
│ null   ┆ 1.210883   ┆ 40.879167 ┆ -111.85518 ┆ … ┆ null      ┆ null      ┆ null      ┆ null      │
│        ┆            ┆           ┆ 1          ┆   ┆           ┆       

In [64]:
# 1.09 Megabytes
df.estimated_size()

1091240

In [65]:
# no index!
df

course,distance_2d,latitude,longitude,time,elevation,speed_between,travelled,elapsed,avg_velocity,rolling_travelled,rolling_elapsed,rolling_velocity,rolling_between
null,f64,f64,f64,"datetime[μs, UTC]",f64,f64,f64,i64,f64,f64,f64,f64,f64
null,0.0,40.879161,-111.855169,2024-09-10 23:41:56 UTC,1480.0,null,0.0,0,NaN,null,null,null,null
null,1.210883,40.879167,-111.855181,2024-09-10 23:41:58 UTC,1480.1,0.607503,1.210883,2,0.605442,null,null,null,null
null,1.227612,40.879172,-111.855194,2024-09-10 23:41:59 UTC,1480.1,1.227612,2.438495,3,0.812832,null,null,null,null
null,0.952238,40.879174,-111.855205,2024-09-10 23:42:00 UTC,1480.1,0.952238,3.390733,4,0.847683,null,null,null,null
null,0.90551,40.879177,-111.855215,2024-09-10 23:42:01 UTC,1480.1,0.90551,4.296243,5,0.859249,2.267271,2.8,0.80974,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…
null,4.085147,40.857847,-111.823899,2024-09-11 02:39:58 UTC,1696.8,4.086371,18640.85135,10682,1.745071,18634.60004,10680.0,1.744813,3.177668
null,1.535726,40.857852,-111.823916,2024-09-11 02:39:59 UTC,1696.8,1.535726,18642.387076,10683,1.745052,18637.420852,10681.0,1.744913,2.821613
null,3.156134,40.857869,-111.823946,2024-09-11 02:40:00 UTC,1696.7,3.157718,18645.54321,10684,1.745184,18640.220738,10682.0,1.745012,2.800696


In [66]:
(df.with_row_index())

index,course,distance_2d,latitude,longitude,time,elevation,speed_between,travelled,elapsed,avg_velocity,rolling_travelled,rolling_elapsed,rolling_velocity,rolling_between
u32,null,f64,f64,f64,"datetime[μs, UTC]",f64,f64,f64,i64,f64,f64,f64,f64,f64
0,null,0.0,40.879161,-111.855169,2024-09-10 23:41:56 UTC,1480.0,null,0.0,0,NaN,null,null,null,null
1,null,1.210883,40.879167,-111.855181,2024-09-10 23:41:58 UTC,1480.1,0.607503,1.210883,2,0.605442,null,null,null,null
2,null,1.227612,40.879172,-111.855194,2024-09-10 23:41:59 UTC,1480.1,1.227612,2.438495,3,0.812832,null,null,null,null
3,null,0.952238,40.879174,-111.855205,2024-09-10 23:42:00 UTC,1480.1,0.952238,3.390733,4,0.847683,null,null,null,null
4,null,0.90551,40.879177,-111.855215,2024-09-10 23:42:01 UTC,1480.1,0.90551,4.296243,5,0.859249,2.267271,2.8,0.80974,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
10425,null,4.085147,40.857847,-111.823899,2024-09-11 02:39:58 UTC,1696.8,4.086371,18640.85135,10682,1.745071,18634.60004,10680.0,1.744813,3.177668
10426,null,1.535726,40.857852,-111.823916,2024-09-11 02:39:59 UTC,1696.8,1.535726,18642.387076,10683,1.745052,18637.420852,10681.0,1.744913,2.821613
10427,null,3.156134,40.857869,-111.823946,2024-09-11 02:40:00 UTC,1696.7,3.157718,18645.54321,10684,1.745184,18640.220738,10682.0,1.745012,2.800696


## Types
Getting the right types will enable analysis and correctness.

In [67]:
print(df.columns)

['course', 'distance_2d', 'latitude', 'longitude', 'time', 'elevation', 'speed_between', 'travelled', 'elapsed', 'avg_velocity', 'rolling_travelled', 'rolling_elapsed', 'rolling_velocity', 'rolling_between']


In [68]:
cols = ['course', 'distance_2d', 'latitude', 'longitude', 'time', 'elevation', 'speed_between', 'travelled', 
        'elapsed', 'avg_velocity', 'rolling_travelled', 'rolling_elapsed', 'rolling_velocity', 'rolling_between']

In [69]:
df[cols].dtypes

[Null,
 Float64,
 Float64,
 Float64,
 Datetime(time_unit='us', time_zone='UTC'),
 Float64,
 Float64,
 Float64,
 Int64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64]

In [70]:
# 1.09 Megabytes
df[cols].estimated_size()

1091240

### Ints

In [71]:
# select is a context
# pl.col is an expression
df[cols].select(pl.col(pl.Int64))

elapsed
i64
0
2
3
4
5
…
10682
10683
10684


In [72]:
df[cols].select(pl.col(pl.Int64)).describe()

statistic,elapsed
str,f64
"""count""",10430.0
"""null_count""",0.0
"""mean""",5403.773921
"""std""",3067.406074
"""min""",0.0
"""25%""",2760.0
"""50%""",5424.0
"""75%""",8069.0
"""max""",10686.0


In [76]:
# chaining
(df
 .select(cols)
 .select(pl.col(pl.Int64))
 .describe()
)

statistic,elapsed
str,f64
"""count""",10430.0
"""null_count""",0.0
"""mean""",5403.773921
"""std""",3067.406074
"""min""",0.0
"""25%""",2760.0
"""50%""",5424.0
"""75%""",8069.0
"""max""",10686.0


In [77]:
# can elapsed be an int8?
np.iinfo(np.int8)

iinfo(min=-128, max=127, dtype=int8)

In [78]:
# can elapsed be an int16?
np.iinfo(np.int16)

iinfo(min=-32768, max=32767, dtype=int16)

In [79]:
# chaining
# polars prevents illegal casts
(df
 .select(cols)
 .with_columns(pl.col('elapsed').cast(pl.Int8))
 #.describe()
)

InvalidOperationError: conversion from `i64` to `i8` failed in column 'elapsed' for 10321 out of 10430 values: [128, 129, … 10686]

In [80]:
# chaining
(df
 .select(cols)
 .with_columns(pl.col('elapsed').cast(pl.Int16))
 .describe()               
)

statistic,course,distance_2d,latitude,longitude,time,elevation,speed_between,travelled,elapsed,avg_velocity,rolling_travelled,rolling_elapsed,rolling_velocity,rolling_between
str,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",0.0,10430.0,10430.0,10430.0,"""10430""",10430.0,10429.0,10430.0,10430.0,10430.0,10426.0,10426.0,10426.0,10425.0
"""null_count""",10430.0,0.0,0.0,0.0,"""0""",0.0,1.0,0.0,0.0,0.0,4.0,4.0,4.0,5.0
"""mean""",null,1.788392,40.859662,-111.822538,"""2024-09-11 01:11:59.773921+00:…",1788.95372,1.811534,10644.58751,5403.773921,NaN,10645.09369,5403.797123,2.065183,1.81141
"""std""",null,1.710749,0.005536,0.0137,null,141.492882,1.657749,5393.503724,3067.406074,NaN,5391.382656,3066.209114,0.418829,1.506718
"""min""",null,0.0,40.848371,-111.855371,"""2024-09-10 23:41:56+00:00""",1480.0,0.0,0.0,0.0,0.374262,2.267271,2.8,0.376911,-1.0081e-14
"""25%""",null,0.611231,40.856748,-111.829785,"""2024-09-11 00:27:56+00:00""",1696.0,0.675361,5810.431187,2760.0,1.848393,5812.456847,2761.0,1.848439,0.776866
"""50%""",null,1.324637,40.859156,-111.82222,"""2024-09-11 01:12:20+00:00""",1816.5,1.392097,12239.423704,5424.0,1.981256,12239.47112,5424.0,1.981257,1.582638
"""75%""",null,2.424472,40.86085,-111.813012,"""2024-09-11 01:56:25+00:00""",1924.9,2.506978,15087.566241,8069.0,2.231917,15086.449322,8068.0,2.231922,2.428585
"""max""",null,15.32443,40.879271,-111.801421,"""2024-09-11 02:40:02+00:00""",1984.0,12.871382,18652.930528,10686.0,3.828152,18646.176372,10684.0,3.827216,12.491514


In [81]:
# chaining
(df
 .select(cols)
 .with_columns(pl.col('elapsed').cast(pl.Int16)) 
 .estimated_size()
)

1028660

In [82]:
(df
 .select(cols)
 .estimated_size()
)

1091240

### Strings

In [83]:
df.select(cols).select(pl.col(pl.String))

shape: (0, 0)
┌┐
╞╡
└┘

In [84]:
# chaining
(df
 .select(cols)
 .with_columns(pl.col('elapsed').cast(pl.Int16),
               course=pl.lit('Maple Syrup'))
)

course,distance_2d,latitude,longitude,time,elevation,speed_between,travelled,elapsed,avg_velocity,rolling_travelled,rolling_elapsed,rolling_velocity,rolling_between
str,f64,f64,f64,"datetime[μs, UTC]",f64,f64,f64,i16,f64,f64,f64,f64,f64
"""Maple Syrup""",0.0,40.879161,-111.855169,2024-09-10 23:41:56 UTC,1480.0,null,0.0,0,NaN,null,null,null,null
"""Maple Syrup""",1.210883,40.879167,-111.855181,2024-09-10 23:41:58 UTC,1480.1,0.607503,1.210883,2,0.605442,null,null,null,null
"""Maple Syrup""",1.227612,40.879172,-111.855194,2024-09-10 23:41:59 UTC,1480.1,1.227612,2.438495,3,0.812832,null,null,null,null
"""Maple Syrup""",0.952238,40.879174,-111.855205,2024-09-10 23:42:00 UTC,1480.1,0.952238,3.390733,4,0.847683,null,null,null,null
"""Maple Syrup""",0.90551,40.879177,-111.855215,2024-09-10 23:42:01 UTC,1480.1,0.90551,4.296243,5,0.859249,2.267271,2.8,0.80974,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Maple Syrup""",4.085147,40.857847,-111.823899,2024-09-11 02:39:58 UTC,1696.8,4.086371,18640.85135,10682,1.745071,18634.60004,10680.0,1.744813,3.177668
"""Maple Syrup""",1.535726,40.857852,-111.823916,2024-09-11 02:39:59 UTC,1696.8,1.535726,18642.387076,10683,1.745052,18637.420852,10681.0,1.744913,2.821613
"""Maple Syrup""",3.156134,40.857869,-111.823946,2024-09-11 02:40:00 UTC,1696.7,3.157718,18645.54321,10684,1.745184,18640.220738,10682.0,1.745012,2.800696


In [85]:
# an example of a string operation
(df
 .select(cols)
 .with_columns(pl.col('elapsed').cast(pl.Int16),
               course=pl.lit('Maple Syrup'))
 .with_columns(alt_name=pl.col('course').str.to_uppercase()) 
 
)

course,distance_2d,latitude,longitude,time,elevation,speed_between,travelled,elapsed,avg_velocity,rolling_travelled,rolling_elapsed,rolling_velocity,rolling_between,alt_name
str,f64,f64,f64,"datetime[μs, UTC]",f64,f64,f64,i16,f64,f64,f64,f64,f64,str
"""Maple Syrup""",0.0,40.879161,-111.855169,2024-09-10 23:41:56 UTC,1480.0,null,0.0,0,NaN,null,null,null,null,"""MAPLE SYRUP"""
"""Maple Syrup""",1.210883,40.879167,-111.855181,2024-09-10 23:41:58 UTC,1480.1,0.607503,1.210883,2,0.605442,null,null,null,null,"""MAPLE SYRUP"""
"""Maple Syrup""",1.227612,40.879172,-111.855194,2024-09-10 23:41:59 UTC,1480.1,1.227612,2.438495,3,0.812832,null,null,null,null,"""MAPLE SYRUP"""
"""Maple Syrup""",0.952238,40.879174,-111.855205,2024-09-10 23:42:00 UTC,1480.1,0.952238,3.390733,4,0.847683,null,null,null,null,"""MAPLE SYRUP"""
"""Maple Syrup""",0.90551,40.879177,-111.855215,2024-09-10 23:42:01 UTC,1480.1,0.90551,4.296243,5,0.859249,2.267271,2.8,0.80974,null,"""MAPLE SYRUP"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Maple Syrup""",4.085147,40.857847,-111.823899,2024-09-11 02:39:58 UTC,1696.8,4.086371,18640.85135,10682,1.745071,18634.60004,10680.0,1.744813,3.177668,"""MAPLE SYRUP"""
"""Maple Syrup""",1.535726,40.857852,-111.823916,2024-09-11 02:39:59 UTC,1696.8,1.535726,18642.387076,10683,1.745052,18637.420852,10681.0,1.744913,2.821613,"""MAPLE SYRUP"""
"""Maple Syrup""",3.156134,40.857869,-111.823946,2024-09-11 02:40:00 UTC,1696.7,3.157718,18645.54321,10684,1.745184,18640.220738,10682.0,1.745012,2.800696,"""MAPLE SYRUP"""


In [86]:
# a bunch of string methods off of .str
# note that the spelling might be different from python/pandas
col = pl.col('')
print(dir(col.str))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_accessor', '_pyexpr', 'concat', 'contains', 'contains_any', 'count_matches', 'decode', 'encode', 'ends_with', 'escape_regex', 'explode', 'extract', 'extract_all', 'extract_groups', 'extract_many', 'find', 'head', 'join', 'json_decode', 'json_path_match', 'len_bytes', 'len_chars', 'pad_end', 'pad_start', 'replace', 'replace_all', 'replace_many', 'reverse', 'slice', 'split', 'split_exact', 'splitn', 'starts_with', 'strip_chars', 'strip_chars_end', 'strip_chars_start', 'strip_prefix', 'strip_suffix', 'strptime', 'tail', 'to_date', 'to_datetime', 'to_decimal', 'to_integer', 'to_lowercase', 'to_time', 'to_titlecase', 'to_uppercase', 'zfill']


In [87]:
col.str.to_uppercase?

## Convert Date to Local Time

In [88]:
(df
 .select(cols)
 .with_columns(pl.col('elapsed').cast(pl.Int16),
               course=pl.lit('Maple Syrup').cast(pl.Categorical),
               time=pl.col('time').dt.convert_time_zone('America/Denver')
              )
)

course,distance_2d,latitude,longitude,time,elevation,speed_between,travelled,elapsed,avg_velocity,rolling_travelled,rolling_elapsed,rolling_velocity,rolling_between
cat,f64,f64,f64,"datetime[μs, America/Denver]",f64,f64,f64,i16,f64,f64,f64,f64,f64
"""Maple Syrup""",0.0,40.879161,-111.855169,2024-09-10 17:41:56 MDT,1480.0,null,0.0,0,NaN,null,null,null,null
"""Maple Syrup""",1.210883,40.879167,-111.855181,2024-09-10 17:41:58 MDT,1480.1,0.607503,1.210883,2,0.605442,null,null,null,null
"""Maple Syrup""",1.227612,40.879172,-111.855194,2024-09-10 17:41:59 MDT,1480.1,1.227612,2.438495,3,0.812832,null,null,null,null
"""Maple Syrup""",0.952238,40.879174,-111.855205,2024-09-10 17:42:00 MDT,1480.1,0.952238,3.390733,4,0.847683,null,null,null,null
"""Maple Syrup""",0.90551,40.879177,-111.855215,2024-09-10 17:42:01 MDT,1480.1,0.90551,4.296243,5,0.859249,2.267271,2.8,0.80974,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Maple Syrup""",4.085147,40.857847,-111.823899,2024-09-10 20:39:58 MDT,1696.8,4.086371,18640.85135,10682,1.745071,18634.60004,10680.0,1.744813,3.177668
"""Maple Syrup""",1.535726,40.857852,-111.823916,2024-09-10 20:39:59 MDT,1696.8,1.535726,18642.387076,10683,1.745052,18637.420852,10681.0,1.744913,2.821613
"""Maple Syrup""",3.156134,40.857869,-111.823946,2024-09-10 20:40:00 MDT,1696.7,3.157718,18645.54321,10684,1.745184,18640.220738,10682.0,1.745012,2.800696


In [89]:
col = pl.col('time')
print(dir(col))

['__abs__', '__add__', '__and__', '__annotations__', '__array_ufunc__', '__bool__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__floordiv__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__invert__', '__le__', '__lt__', '__mod__', '__module__', '__mul__', '__ne__', '__neg__', '__new__', '__or__', '__pos__', '__pow__', '__radd__', '__rand__', '__reduce__', '__reduce_ex__', '__repr__', '__rfloordiv__', '__rmod__', '__rmul__', '__ror__', '__rpow__', '__rsub__', '__rtruediv__', '__rxor__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__truediv__', '__weakref__', '__xor__', '_accessors', '_from_pyexpr', '_map_batches_wrapper', '_pyexpr', '_repr_html_', 'abs', 'add', 'agg_groups', 'alias', 'all', 'and_', 'any', 'append', 'approx_n_unique', 'arccos', 'arccosh', 'arcsin', 'arcsinh', 'arctan', 'arctanh', 'arg_max', 'arg_min', 'arg_sort', 'arg_true', 'ar

In [90]:
print(len(dir(col)))

268


In [91]:
print(dir(col.dt))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_accessor', '_pyexpr', 'add_business_days', 'base_utc_offset', 'cast_time_unit', 'century', 'combine', 'convert_time_zone', 'date', 'datetime', 'day', 'dst_offset', 'epoch', 'hour', 'is_leap_year', 'iso_year', 'microsecond', 'millennium', 'millisecond', 'minute', 'month', 'month_end', 'month_start', 'nanosecond', 'offset_by', 'ordinal_day', 'quarter', 'replace_time_zone', 'round', 'second', 'strftime', 'time', 'timestamp', 'to_string', 'total_days', 'total_hours', 'total_microseconds', 'total_milliseconds', 'total_minutes', 'total_nanoseconds', 'total_seconds', 'truncate', 'week', 'weekday', 'with_time_unit', 'year']


In [92]:
print(len(dir(col.dt)))

72


## Missing Data

- Use `.fill_null` to address
- Use `.filter` to filter rows
- Use `.select` to select columns

To view rows with missing data use `.filter(pl.col("col_name").is_null())`

In [93]:
(df
 .select(cols)
 .with_columns(pl.col('elapsed').cast(pl.Int16),
               course=pl.lit('Maple Syrup').cast(pl.Categorical),
               time=pl.col('time').dt.convert_time_zone('America/Denver'))
 .null_count()
)

course,distance_2d,latitude,longitude,time,elevation,speed_between,travelled,elapsed,avg_velocity,rolling_travelled,rolling_elapsed,rolling_velocity,rolling_between
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,1,0,0,0,4,4,4,5


In [94]:
# use .select to find where rows are missing
(df
 .select(cols)
 .with_columns(pl.col('elapsed').cast(pl.Int16),
               course=pl.lit('Maple Syrup').cast(pl.Categorical),
               time=pl.col('time').dt.convert_time_zone('America/Denver'))
 .select(pl.col('rolling_between').is_null())
)

rolling_between
bool
true
true
true
true
true
…
false
false
false


In [95]:
# change .select to .filter to view the rows
(df
 .select(cols)
 .with_row_index()
 .with_columns(pl.col('elapsed').cast(pl.Int16),
               course=pl.lit('Maple Syrup').cast(pl.Categorical),
               time=pl.col('time').dt.convert_time_zone('America/Denver'))               
 .filter(pl.col('rolling_between').is_null())
)

index,course,distance_2d,latitude,longitude,time,elevation,speed_between,travelled,elapsed,avg_velocity,rolling_travelled,rolling_elapsed,rolling_velocity,rolling_between
u32,cat,f64,f64,f64,"datetime[μs, America/Denver]",f64,f64,f64,i16,f64,f64,f64,f64,f64
0,"""Maple Syrup""",0.0,40.879161,-111.855169,2024-09-10 17:41:56 MDT,1480.0,null,0.0,0,NaN,null,null,null,null
1,"""Maple Syrup""",1.210883,40.879167,-111.855181,2024-09-10 17:41:58 MDT,1480.1,0.607503,1.210883,2,0.605442,null,null,null,null
2,"""Maple Syrup""",1.227612,40.879172,-111.855194,2024-09-10 17:41:59 MDT,1480.1,1.227612,2.438495,3,0.812832,null,null,null,null
3,"""Maple Syrup""",0.952238,40.879174,-111.855205,2024-09-10 17:42:00 MDT,1480.1,0.952238,3.390733,4,0.847683,null,null,null,null
4,"""Maple Syrup""",0.90551,40.879177,-111.855215,2024-09-10 17:42:01 MDT,1480.1,0.90551,4.296243,5,0.859249,2.267271,2.8,0.80974,null


In [96]:
# what about nans?
# note that nan and null are different in polars
# nan means not a number
# null means missing data
(df
 .select(cols)
 .with_columns(pl.col('elapsed').cast(pl.Int16),
               course=pl.lit('Maple Syrup').cast(pl.Categorical),
               time=pl.col('time').dt.convert_time_zone('America/Denver'))               
 .select(cs.numeric().is_nan().sum())
)

distance_2d,latitude,longitude,elevation,speed_between,travelled,elapsed,avg_velocity,rolling_travelled,rolling_elapsed,rolling_velocity,rolling_between
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,1,0,0,0,0


In [97]:
(df
 .select(cols)
 .with_columns(pl.col('elapsed').cast(pl.Int16),
               course=pl.lit('Maple Syrup').cast(pl.Categorical),
               time=pl.col('time').dt.convert_time_zone('America/Denver'))               
 .select(pl.col('avg_velocity').is_nan())
)

avg_velocity
bool
true
false
false
false
false
…
false
false
false


In [98]:
(df
 .select(cols)
 .with_row_index()  
 .with_columns(pl.col('elapsed').cast(pl.Int16),
               course=pl.lit('Maple Syrup').cast(pl.Categorical),
               time=pl.col('time').dt.convert_time_zone('America/Denver'))               
 .filter(pl.col('avg_velocity').is_nan())
)

index,course,distance_2d,latitude,longitude,time,elevation,speed_between,travelled,elapsed,avg_velocity,rolling_travelled,rolling_elapsed,rolling_velocity,rolling_between
u32,cat,f64,f64,f64,"datetime[μs, America/Denver]",f64,f64,f64,i16,f64,f64,f64,f64,f64
0,"""Maple Syrup""",0.0,40.879161,-111.855169,2024-09-10 17:41:56 MDT,1480.0,null,0.0,0,NaN,null,null,null,null


In [99]:
# a glorious function

def tweak_gpx(df_):
    return (df_
        .select(cols)
        .with_row_index()  
        .with_columns(pl.col('elapsed').cast(pl.Int16),
                    course=pl.lit('Maple Syrup').cast(pl.Categorical),
                    time=pl.col('time').dt.convert_time_zone('America/Denver'))               
        )

tweak_gpx(df)

index,course,distance_2d,latitude,longitude,time,elevation,speed_between,travelled,elapsed,avg_velocity,rolling_travelled,rolling_elapsed,rolling_velocity,rolling_between
u32,cat,f64,f64,f64,"datetime[μs, America/Denver]",f64,f64,f64,i16,f64,f64,f64,f64,f64
0,"""Maple Syrup""",0.0,40.879161,-111.855169,2024-09-10 17:41:56 MDT,1480.0,null,0.0,0,NaN,null,null,null,null
1,"""Maple Syrup""",1.210883,40.879167,-111.855181,2024-09-10 17:41:58 MDT,1480.1,0.607503,1.210883,2,0.605442,null,null,null,null
2,"""Maple Syrup""",1.227612,40.879172,-111.855194,2024-09-10 17:41:59 MDT,1480.1,1.227612,2.438495,3,0.812832,null,null,null,null
3,"""Maple Syrup""",0.952238,40.879174,-111.855205,2024-09-10 17:42:00 MDT,1480.1,0.952238,3.390733,4,0.847683,null,null,null,null
4,"""Maple Syrup""",0.90551,40.879177,-111.855215,2024-09-10 17:42:01 MDT,1480.1,0.90551,4.296243,5,0.859249,2.267271,2.8,0.80974,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
10425,"""Maple Syrup""",4.085147,40.857847,-111.823899,2024-09-10 20:39:58 MDT,1696.8,4.086371,18640.85135,10682,1.745071,18634.60004,10680.0,1.744813,3.177668
10426,"""Maple Syrup""",1.535726,40.857852,-111.823916,2024-09-10 20:39:59 MDT,1696.8,1.535726,18642.387076,10683,1.745052,18637.420852,10681.0,1.744913,2.821613
10427,"""Maple Syrup""",3.156134,40.857869,-111.823946,2024-09-10 20:40:00 MDT,1696.7,3.157718,18645.54321,10684,1.745184,18640.220738,10682.0,1.745012,2.800696


## Chain

Chaining is also called "flow" programming. Rather than making intermediate variables, just leverage the fact that most operations return a new object and work on that.

The chain should read like a recipe of ordered steps.

(BTW, this is actually what we did above.)

In [100]:
# a glorious function

def tweak_gpx(df_):
    return (df_
        .select(cols)
        .with_row_index()  
        .with_columns(pl.col('elapsed').cast(pl.Int16),
                    course=pl.lit('Maple Syrup').cast(pl.Categorical),
                    time=pl.col('time').dt.convert_time_zone('America/Denver'))               
        )

tweak_gpx(df).write_parquet('Face_plant.parquet')

In [101]:
# laziness
gpx_lazy = pl.scan_parquet('Face_plant.parquet') 
tweak_gpx(gpx_lazy)

In [102]:
# use .collect to generate plan and materialize
tweak_gpx(gpx_lazy).collect()

index,course,distance_2d,latitude,longitude,time,elevation,speed_between,travelled,elapsed,avg_velocity,rolling_travelled,rolling_elapsed,rolling_velocity,rolling_between
u32,cat,f64,f64,f64,"datetime[μs, America/Denver]",f64,f64,f64,i16,f64,f64,f64,f64,f64
0,"""Maple Syrup""",0.0,40.879161,-111.855169,2024-09-10 17:41:56 MDT,1480.0,null,0.0,0,NaN,null,null,null,null
1,"""Maple Syrup""",1.210883,40.879167,-111.855181,2024-09-10 17:41:58 MDT,1480.1,0.607503,1.210883,2,0.605442,null,null,null,null
2,"""Maple Syrup""",1.227612,40.879172,-111.855194,2024-09-10 17:41:59 MDT,1480.1,1.227612,2.438495,3,0.812832,null,null,null,null
3,"""Maple Syrup""",0.952238,40.879174,-111.855205,2024-09-10 17:42:00 MDT,1480.1,0.952238,3.390733,4,0.847683,null,null,null,null
4,"""Maple Syrup""",0.90551,40.879177,-111.855215,2024-09-10 17:42:01 MDT,1480.1,0.90551,4.296243,5,0.859249,2.267271,2.8,0.80974,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
10425,"""Maple Syrup""",4.085147,40.857847,-111.823899,2024-09-10 20:39:58 MDT,1696.8,4.086371,18640.85135,10682,1.745071,18634.60004,10680.0,1.744813,3.177668
10426,"""Maple Syrup""",1.535726,40.857852,-111.823916,2024-09-10 20:39:59 MDT,1696.8,1.535726,18642.387076,10683,1.745052,18637.420852,10681.0,1.744913,2.821613
10427,"""Maple Syrup""",3.156134,40.857869,-111.823946,2024-09-10 20:40:00 MDT,1696.7,3.157718,18645.54321,10684,1.745184,18640.220738,10682.0,1.745012,2.800696


In [103]:
# using GPU!
tweak_gpx(gpx_lazy).collect('gpu')

TypeError: LazyFrame.collect() takes 1 positional argument but 2 were given

In [104]:
# debugging
# some folks really want the intermediate data...
def get_var(df, var_name):
   globals()[var_name] = df
   return df

def tweak_gpx(df_):
    return (df_
        .pipe(lambda df: print(df.shape) or df)  # Look! 🤯
        .select(cols)
        .with_row_index()  
        .pipe(get_var, 'intermediate')  # Debugging! 💪
        .with_columns(pl.col('elapsed').cast(pl.Int16),
                    course=pl.lit('Maple Syrup').cast(pl.Categorical),
                    time=pl.col('time').dt.convert_time_zone('America/Denver'))               
        )

raw = pl.read_parquet('Face_plant.parquet')
tweak_gpx(raw)

(10430, 15)


index,course,distance_2d,latitude,longitude,time,elevation,speed_between,travelled,elapsed,avg_velocity,rolling_travelled,rolling_elapsed,rolling_velocity,rolling_between
u32,cat,f64,f64,f64,"datetime[μs, America/Denver]",f64,f64,f64,i16,f64,f64,f64,f64,f64
0,"""Maple Syrup""",0.0,40.879161,-111.855169,2024-09-10 17:41:56 MDT,1480.0,null,0.0,0,NaN,null,null,null,null
1,"""Maple Syrup""",1.210883,40.879167,-111.855181,2024-09-10 17:41:58 MDT,1480.1,0.607503,1.210883,2,0.605442,null,null,null,null
2,"""Maple Syrup""",1.227612,40.879172,-111.855194,2024-09-10 17:41:59 MDT,1480.1,1.227612,2.438495,3,0.812832,null,null,null,null
3,"""Maple Syrup""",0.952238,40.879174,-111.855205,2024-09-10 17:42:00 MDT,1480.1,0.952238,3.390733,4,0.847683,null,null,null,null
4,"""Maple Syrup""",0.90551,40.879177,-111.855215,2024-09-10 17:42:01 MDT,1480.1,0.90551,4.296243,5,0.859249,2.267271,2.8,0.80974,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
10425,"""Maple Syrup""",4.085147,40.857847,-111.823899,2024-09-10 20:39:58 MDT,1696.8,4.086371,18640.85135,10682,1.745071,18634.60004,10680.0,1.744813,3.177668
10426,"""Maple Syrup""",1.535726,40.857852,-111.823916,2024-09-10 20:39:59 MDT,1696.8,1.535726,18642.387076,10683,1.745052,18637.420852,10681.0,1.744913,2.821613
10427,"""Maple Syrup""",3.156134,40.857869,-111.823946,2024-09-10 20:40:00 MDT,1696.7,3.157718,18645.54321,10684,1.745184,18640.220738,10682.0,1.745012,2.800696


In [105]:
intermediate

index,course,distance_2d,latitude,longitude,time,elevation,speed_between,travelled,elapsed,avg_velocity,rolling_travelled,rolling_elapsed,rolling_velocity,rolling_between
u32,cat,f64,f64,f64,"datetime[μs, America/Denver]",f64,f64,f64,i16,f64,f64,f64,f64,f64
0,"""Maple Syrup""",0.0,40.879161,-111.855169,2024-09-10 17:41:56 MDT,1480.0,null,0.0,0,NaN,null,null,null,null
1,"""Maple Syrup""",1.210883,40.879167,-111.855181,2024-09-10 17:41:58 MDT,1480.1,0.607503,1.210883,2,0.605442,null,null,null,null
2,"""Maple Syrup""",1.227612,40.879172,-111.855194,2024-09-10 17:41:59 MDT,1480.1,1.227612,2.438495,3,0.812832,null,null,null,null
3,"""Maple Syrup""",0.952238,40.879174,-111.855205,2024-09-10 17:42:00 MDT,1480.1,0.952238,3.390733,4,0.847683,null,null,null,null
4,"""Maple Syrup""",0.90551,40.879177,-111.855215,2024-09-10 17:42:01 MDT,1480.1,0.90551,4.296243,5,0.859249,2.267271,2.8,0.80974,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
10425,"""Maple Syrup""",4.085147,40.857847,-111.823899,2024-09-10 20:39:58 MDT,1696.8,4.086371,18640.85135,10682,1.745071,18634.60004,10680.0,1.744813,3.177668
10426,"""Maple Syrup""",1.535726,40.857852,-111.823916,2024-09-10 20:39:59 MDT,1696.8,1.535726,18642.387076,10683,1.745052,18637.420852,10681.0,1.744913,2.821613
10427,"""Maple Syrup""",3.156134,40.857869,-111.823946,2024-09-10 20:40:00 MDT,1696.7,3.157718,18645.54321,10684,1.745184,18640.220738,10682.0,1.745012,2.800696


## Don't Apply (map_elements) if you can

In [106]:
# debugging
def get_var(df, var_name):
   globals()[var_name] = df
   return df

def tweak_gpx(df_):
    return (df_
             .pipe(lambda df: print(df.shape) or df)
        .select(cols)
        .with_row_index()  
         .pipe(get_var, 'intermediate')
        .with_columns(pl.col('elapsed').cast(pl.Int16),
                    course=pl.lit('Maple Syrup').cast(pl.Categorical),
                    time=pl.col('time').dt.convert_time_zone('America/Denver'))               
        )

raw = pl.read_parquet('Face_plant.parquet')
df = tweak_gpx(raw)

(10430, 15)


In [107]:
# convert elevation from meters to feet
def meters_to_feet(m):
    return m * 3.28084

(df
 .select('elevation', 
         ele_ft=pl.col('elevation').map_elements(meters_to_feet)) 
)

/var/folders/qn/r8_0pgj1645dn1w69vqls6cw0000gn/T/ipykernel_70391/613516876.py:7: PolarsInefficientMapWarning: 
Expr.map_elements is significantly slower than the native expressions API.
Only use if you absolutely CANNOT implement your logic otherwise.
Replace this expression...
  - pl.col("elevation").map_elements(meters_to_feet)
with this one instead:
  + pl.col("elevation") * 3.28084

  ele_ft=pl.col('elevation').map_elements(meters_to_feet))
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.


elevation,ele_ft
f64,f64
1480.0,4855.6432
1480.1,4855.971284
1480.1,4855.971284
1480.1,4855.971284
1480.1,4855.971284
…,…
1696.8,5566.929312
1696.8,5566.929312
1696.7,5566.601228


In [108]:
# convert elevation from meters to feet
def meters_to_feet(m):
    return m * 3.28084

(df
 .select('elevation', 
         ele_ft=meters_to_feet(pl.col('elevation')))
)

elevation,ele_ft
f64,f64
1480.0,4855.6432
1480.1,4855.971284
1480.1,4855.971284
1480.1,4855.971284
1480.1,4855.971284
…,…
1696.8,5566.929312
1696.8,5566.929312
1696.7,5566.601228


In [109]:
# Perhaps more readable
# convert elevation from meters to feet
def meters_to_feet(m):
    return m * 3.28084

(df
 .select('elevation', 
         ele_ft=pl.col('elevation').pipe(meters_to_feet))
)

elevation,ele_ft
f64,f64
1480.0,4855.6432
1480.1,4855.971284
1480.1,4855.971284
1480.1,4855.971284
1480.1,4855.971284
…,…
1696.8,5566.929312
1696.8,5566.929312
1696.7,5566.601228


In [ ]:
%%timeit
# takes 965 µs on my machine
(df
 .select('elevation', ele_ft=pl.col('elevation').map_elements(meters_to_feet)) 
)

In [110]:
%%timeit
(df
 .select('elevation', 
         ele_ft=pl.col('elevation').pipe(meters_to_feet))
)

38 µs ± 514 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [111]:
%%timeit
(df
 .select('elevation', 
         ele_ft=pl.col('elevation')*3.28084)
)

37.7 µs ± 1.11 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [112]:
965/40

24.125

## benchmark caveat
- Use the size of data you are using in the real world

## Master Aggregation

Let's speed (and distance) by 10 minute intervals

In [113]:
def meters_per_second_to_mph(mps):
    return mps * 2.23694

(tweak_gpx(raw)
 .group_by_dynamic(index_column='time', every='10m')
 .agg(pl.col('travelled').last() - pl.col('travelled').first(),
      speed=(pl.col('travelled').last() - pl.col('travelled').first()) / 
          ((pl.col('time').last() - pl.col('time').first()).dt.total_seconds())
    ) 
 .with_columns(mph=pl.col('speed').pipe(meters_per_second_to_mph))
 )

(10430, 15)


time,travelled,speed,mph
"datetime[μs, America/Denver]",f64,f64,f64
2024-09-10 17:40:00 MDT,1620.956003,3.356017,7.507208
2024-09-10 17:50:00 MDT,1081.973828,1.8063,4.040585
2024-09-10 18:00:00 MDT,99.724838,0.166486,0.372418
2024-09-10 18:10:00 MDT,1703.006885,2.843083,6.359807
2024-09-10 18:20:00 MDT,1602.514698,2.675317,5.984523
…,…,…,…
2024-09-10 20:00:00 MDT,973.953159,1.625965,3.637187
2024-09-10 20:10:00 MDT,680.118068,1.135422,2.539872
2024-09-10 20:20:00 MDT,796.69377,1.33004,2.975219


In [114]:
def meters_per_second_to_mph(mps):
    return mps * 2.23694

(tweak_gpx(raw)
 .group_by_dynamic(index_column='time', every='10m')
 .agg(pl.col('travelled').last() - pl.col('travelled').first(),
      speed=(pl.col('travelled').last() - pl.col('travelled').first()) / 
      ((pl.col('time').last() - pl.col('time').first()).dt.total_seconds())
    ) 
 .with_columns(mph=pl.col('speed').pipe(meters_per_second_to_mph))
 .plot.bar(x='time', y='mph')
 )

(10430, 15)


alt.Chart(...)

In [115]:
# uphill vs downhill
def meters_to_feet(m):
    return m * 3.28084

def feet_to_miles(f):
    return f / 5280

(tweak_gpx(raw)
 .with_columns(climbing=pl.col('elevation').diff().gt(0))
 .group_by('climbing')
 .agg(pl.col('distance_2d').sum().pipe(meters_to_feet).pipe(feet_to_miles))
 .filter(~pl.col('climbing').is_null())
)

(10430, 15)


climbing,distance_2d
bool,f64
false,5.46702
true,6.123374


In [116]:
# uphill vs downhill
def meters_to_feet(m):
    return m * 3.28084

def feet_to_miles(f):
    return f / 5280

(tweak_gpx(raw)
 .with_columns(climbing=pl.col('elevation').diff().gt(0))
 .group_by('climbing')
 .agg(pl.col('distance_2d').sum().pipe(meters_to_feet).pipe(feet_to_miles))
 .filter(~pl.col('climbing').is_null())
 .plot.bar(x='climbing', y='distance_2d')
)

(10430, 15)


alt.Chart(...)

## Summary

* Correct types save space and enable convenient math, string, and date functionality
* Chaining operations will:
   * Make code readable
   * Remove bugs
   * Easier to debug
* ``.map_elements`` is slow for math
* Aggregations are powerful. Play with them until they make sense


Let's connect! Happy to discuss how your team can better leverage tabular technologies.

Twitter ``@__mharrison__``, LinkedIn

Book giveaway

In [117]:
import random

In [118]:
random.randrange(0, 12)

10

In [ ]:
random.randrange(0, 3)

In [ ]:
import random
random.choice([0,1])

In [ ]:
import random
random.randrange(1,4)